## EDA and Preprocessing

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
import xgboost as xgb
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
import sklearn.ensemble as ensemble
from sklearn.linear_model import RidgeCV
from sklearn.ensemble import GradientBoostingRegressor
from netCDF4 import Dataset
import os


### Understanding Data Structure:
Each row in train.csv corresponds with the first dimension in the GEFS data array of 15 subsets. The 11 ensemble members make forecasts at 5 time steps over the 9x16 lat-lon grid surrounding Oklahoma. Each column in train.csv is the total daily solar energy for a particular Mesonet site on a particular day. The locations of the Mesonet sites are specified in station_info.csv. You will have to come up with a method to translate the gridded GEFS forecasts to the station locations. How much of the grid you incorporate into your model is up to you.


In [ ]:
df_train = pd.read_csv('../input/iiitb-ai511-ams-solar-energy-prediction-contest/train.csv')
print(df_train.shape)
df_train.head()


There are 98 stations but only need the 6 required stations.
HINT, IDAB, SLAP, WEST, BESS and ACME

In [ ]:
req_stations=['HINT', 'IDAB', 'SLAP', 'WEST', 'BESS', 'ACME']
df_train=df_train.loc[:,['Date']+req_stations]

In [ ]:
df_test=pd.read_csv('../input/iiitb-ai511-ams-solar-energy-prediction-contest/test.csv')
df_test
##need to convert train into this format

In [ ]:
Dates=np.array([i for i in df_train['Date']]*6,dtype='str')
values=np.array(df_train.iloc[:,1:].unstack())

station_names=np.array([i[0] for i in df_train.iloc[:,1:].unstack().index])
res = [i +'-'+ j for i, j in zip(station_names, Dates)]
train_df=pd.DataFrame()
train_df['Id']=res
train_df['Expected']=values
train_df

In [ ]:
#converting longitude by adding 360
df_stations = pd.read_csv('../input/iiitb-ai511-ams-solar-energy-prediction-contest/station_info.csv')
df_stations.head()

#each station can be mapped to near gefs_datagrid points using nlat,elong

#each station has certain levation. this can be added to train and test data later.
boolean=[]
for i in df_stations.stid:
    if i in req_stations:
        boolean.append(True)
    else:
        boolean.append(False)
df_stations=df_stations[boolean].reset_index(drop=True)
df_stations.elon=(df_stations.elon+360)

df_stations


In [ ]:
#understanding structure of data array of each of the 15 laton_subsets
X=Dataset('../input/iiitb-ai511-ams-solar-energy-prediction-contest/gefs_train_updated/train/apcp_sfc_latlon_subset_19940101_20071231.nc')
print(list(X.variables))
print(list(X.variables.values())[-1][:].shape)


In [ ]:
#use all subsets/predictors
predictors = ['apcp_sfc',
        'dlwrf_sfc',
        'dswrf_sfc',
        'pres_msl',
        'pwat_eatm',
        'spfh_2m',
        'tcdc_eatm',
        'tcolc_eatm',
        'tmax_2m',
        'tmin_2m',
        'tmp_2m',
        'tmp_sfc',
        'ulwrf_sfc',
        'ulwrf_tatm',
        'uswrf_sfc']


In [ ]:

acpc=Dataset("../input/iiitb-ai511-ams-solar-energy-prediction-contest/gefs_train_updated/train/apcp_sfc_latlon_subset_19940101_20071231.nc")
#4018 rows, for each date, 11 ensemble members, 5 time differences, 9*16 lat*long grid.
G=list(acpc.variables.values())[-1][:].reshape(5113,55,9,16)
G=G[:4018]
#taking only values till 2004
G=G.mean(axis=1)


In [ ]:
plt.plot(G.mean(axis=0).mean(axis=1))
#downwards trend with increase in latitude
plt.savefig('lattrend.png')

In [ ]:
plt.plot(G.mean(axis=1).mean(axis=0))
#upward trend with increase in longitude.
plt.savefig('longtrend.png')

In [ ]:
list(acpc.variables)

In [ ]:
plt.plot(list(acpc.variables.values())[-1][:].mean(axis=0).mean(axis=1).mean(axis=1).mean(axis=1))
#mean precipitation of different ensemblers
plt.savefig('ensemblertrend.png')

In [ ]:
plt.plot(list(acpc.variables.values())[-1][:].mean(axis=0).mean(axis=0).mean(axis=1).mean(axis=1))
#average precipitation at different hours
plt.savefig('hourstrend.png')

In [ ]:
list(acpc.variables)

In [ ]:
monthly=[]
for i in range(12):
    monthly.append(np.mean(list(acpc.variables.values())[-1][30*i:30*(i+1)]))
plt.plot(monthly)
plt.savefig('monthlytrend.png')

In [ ]:
yearwise=[]
for i in range(2007-1994):
    yearwise.append(list(acpc.variables.values())[-1][i*365:(i+1)*365].mean())


In [ ]:
plt.plot(yearwise)
plt.xticks(np.arange(0,13),np.arange(1994,2007))
plt.savefig('yearlytrend.png')

In [ ]:
station_wise=[]
for i in range(6):
    station_wise.append(train_df[i*df_train.shape[0]:(i+1)*df_train.shape[0]].Expected.mean())


In [ ]:
plt.plot(station_wise)
plt.xticks(np.arange(0,6),req_stations)
plt.savefig('stationtrend.png')
#station wise expected energy

In [ ]:
#predictor paths=../input/iiitb-ai511-ams-solar-energy-prediction-contest/gefs_train_updated/train/apcp_sfc_latlon_subset_19940101_20071231.nc
#used for getting data from predictors
train_path='../input/iiitb-ai511-ams-solar-energy-prediction-contest/gefs_train_updated/train/'
train_range='_latlon_subset_19940101_20071231.nc'
train_date_range=df_train.shape[0]

test_path='../input/iiitb-ai511-ams-solar-energy-prediction-contest/gefs_test_updated/test/'
test_range='_latlon_subset_20080101_20121130.nc'
test_date_range=int(df_test.shape[0]/6)

def get_lat_long_range(station):
    
    # 16*9 grid for each of the 15 predictors:
    #taking only floor , ceil grid data for each station.
    min_lat=int(np.floor(df_stations[df_stations.stid==station].nlat))-31
    max_lat=int(np.ceil(df_stations[df_stations.stid==station].nlat))-31

    min_long=int(np.floor(df_stations[df_stations.stid==station].elon))-254
    max_long=int(np.ceil(df_stations[df_stations.stid==station].elon))-254
    return (min_lat,max_lat,min_long,max_long)

def get_all_predictors(path, predictors, postfix,station,tt):
    for i, predictor in enumerate(predictors):
        if i == 0:
            X = get_predictor(path, predictor, postfix,station,tt)
        else:
            X_append = get_predictor(path, predictor, postfix,station,tt)
            X = np.hstack((X, X_append))

    return X


def get_predictor(path, predictor, postfix,station,tt):
    if tt=='train':
        X = list(Dataset(os.path.join(path, predictor + postfix)).variables.values())[-1][:4018]
    else:
        X = list(Dataset(os.path.join(path, predictor + postfix)).variables.values())[-1][4018:]
    X = X.reshape(X.shape[0], 55, 9, 16)
    X = np.mean(X, axis=1)
    min_lat,max_lat,min_long,max_long=get_lat_long_range(station)
    X=X[:,min_lat:max_lat+1,min_long:max_long+1]
    X = X.reshape(X.shape[0], np.prod(X.shape[1:]))

    return X

def get_X_train():
    print("Progress: 0%")
    for i,st in enumerate(req_stations):
        if i==0:
            X_train=get_all_predictors(train_path,predictors,train_range,st,'train')
        else:
            X_train=np.vstack([X_train,get_all_predictors(train_path,predictors,train_range,st,'train')])
        print("Got data of station: ",st,"....")
        print("Progress: ",int(((i+1)/6)*100),"%")
    return X_train

def get_X_test():
    print("Progress: 0%")
    for i,st in enumerate(req_stations):
        if i==0:
            X_test=get_all_predictors(train_path,predictors,train_range,st,'test')
        else:
            X_test=np.vstack([X_test,get_all_predictors(train_path,predictors,train_range,st,'test')])
        print("Got data of station: ",st,"....")
        print("Progress: ",int(((i+1)/6)*100),"%")
    return X_test


def pred_to_csv(pred,filename):
    subdf=df_test.copy()
    subdf.rename(columns={'Date':'Id'},inplace=True)
    subdf['Expected']=pred
    subdf.to_csv(filename,index=False)
    


In [ ]:

#Prepare train data for each station-date combinaton
X_train_data=get_X_train()
X_train_data=X_train_data.data
print(X_train_data.shape)
X_test_data=get_X_test()
X_test_data=X_test_data.data
print(X_test_data.shape)

In [ ]:
print(X_test_data.shape)

In [ ]:
station_wise_prec=[]
for i in range(6):
    station_wise_prec.append(X_train_data[i*df_train.shape[0]:(i+1)*df_train.shape[0]].mean())


In [ ]:
plt.plot(station_wise_prec,color='r')
plt.xticks(np.arange(0,6),req_stations)
plt.savefig('stationwiseprec.png')
#station wise average precipitation of nearby grid points

In [ ]:
plt.plot((station_wise-np.mean(station_wise))/np.std(station_wise),color='r')
plt.plot((station_wise_prec-np.mean(station_wise_prec))/np.std(station_wise_prec),color='b')
plt.xticks(np.arange(0,6),req_stations)
plt.savefig('precvsenergy.png')

#opposite trend in precipitation and expected energy!

In [ ]:
plt.scatter(station_wise,station_wise_prec)
#expected_energy vs precipitation
plt.savefig('scatter.png')

In [ ]:
np.corrcoef(station_wise,station_wise_prec)[0,1]
#negative strong correlation!

# Modelling

In [ ]:
#Extra Columns extraction
train_years=pd.to_datetime(df_train['Date'], format='%Y%m%d').dt.year
train_months=pd.to_datetime(df_train['Date'],format='%Y%m%d').dt.month

train_years=np.array(list(train_years)*6).reshape(-1,1)
train_months=np.array(list(train_months)*6).reshape(-1,1)

test_years=pd.to_datetime(df_test['Date'].apply(lambda x:x.split('-')[1])).dt.year
test_months=pd.to_datetime(df_test['Date'].apply(lambda x:x.split('-')[1])).dt.month

test_years=np.array(list(test_years)).reshape(-1,1)
test_months=np.array(list(test_months)).reshape(-1,1)

X_train=np.hstack([train_years,train_months,X_train_data])
X_test=np.hstack([test_years,test_months,X_test_data])


In [ ]:
'''
elevations=Dataset('../input/iiitb-ai511-ams-solar-energy-prediction-contest/gefs_elevations.nc')
j=0
avgofboth=[]
for st in req_stations:
        print("Station ",st," \n")
        min_lat,max_lat,min_lon,max_lon=get_lat_long_range(st)
        j=j+1
        flag=1
        
        control=list(elevations.variables.values())[0][:]
        control=control[min_lat:max_lat+1,min_lon:max_lon+1]
        control=control.reshape(4)
        c=control.data.mean()
        ctrl.append(c)
        
        perturbation=list(elevations.variables.values())[1][:]
        perturbation=perturbation[min_lat:max_lat+1,min_lon:max_lon+1]
        perturbation=perturbation.reshape(4)
        p=perturbation.data.mean()
        perb.append(p)
        avgofboth.append((c+p)/2)

repeat=np.repeat(avgofboth,4018).reshape(-1,1)
X_train=np.hstack([repeat,X_train])

repeat=np.repeat(avgofboth,1095).reshape(-1,1)
X_test=np.hstack([repeat,X_test])

'''

In [ ]:
for i,st in enumerate(req_stations):
    if i==0:
        ExtTrain=np.tile(list(df_stations[df_stations.stid==st].iloc[0,1:]),(X_train.shape[0]//6,1))
        
    else:
        ExtTrain=np.vstack([ExtTrain,np.tile(list(df_stations[df_stations.stid==st].iloc[0,1:]),(X_train.shape[0]//6,1))])
        
for i,st in enumerate(req_stations):
    if i==0:
        ExtTest=np.tile(list(df_stations[df_stations.stid==st].iloc[0,1:]),(X_test.shape[0]//6,1))
        
    else:
        ExtTest=np.vstack([ExtTest,np.tile(list(df_stations[df_stations.stid==st].iloc[0,1:]),(X_test.shape[0]//6,1))])
        
X_train=np.hstack([X_train,ExtTrain])
X_test=np.hstack([X_test,ExtTest])

In [ ]:
Y_train=train_df.Expected

In [ ]:
# for each date-station : 2*2 grid * 15 subsets=60 columns
# 4018 dates * 6 stations= 24108 rows
print(X_train.shape)
print(Y_train.shape)
print(X_test.shape)

In [ ]:
from sklearn.linear_model import LinearRegression
import sklearn
from sklearn.model_selection import cross_validate
eval_metric='neg_root_mean_squared_error'
from sklearn.metrics import mean_squared_error

In [ ]:
# prec_train=X_train_data.mean(axis=1)
# LR=LinearRegression()
# cv_results = cross_validate(LR, prec_train.reshape(-1,1), Y_train, cv=5,scoring=eval_metric)
# print("average rmse: ",-np.mean(cv_results['test_score']))


In [ ]:
# cv_results = cross_validate(LR, X_train, Y_train, cv=5,scoring=eval_metric)
# print("average rmse: ",-np.mean(cv_results['test_score']))
# LR.fit(X_train,Y_train)
# LRpredictions=LR.predict(X_test)
# pred_to_csv(LRpredictions,'./lrsub.csv')

In [ ]:
# ridgemodel = RidgeCV(alphas=np.logspace(-3, 3, 10,base=10), cv=5,scoring=eval_metric)
# cv_results = cross_validate(ridgemodel, X_train, Y_train, cv=5,scoring=eval_metric)
# print("average rmse: ",-np.mean(cv_results['test_score']))
# ridgemodel.fit(X_train,Y_train)
# RidgePredictions=ridgemodel.predict(X_test)
# pred_to_csv(RidgePredictions,'./ridgesub.csv')

In [ ]:
# rfmodel=ensemble.RandomForestRegressor()
# parameters = {'n_estimators': [500,1000]}
# gridCV = GridSearchCV(rfmodel, n_jobs=-1,param_grid=parameters, cv=5,scoring=eval_metric)
# gridCV.fit(X_train,Y_train)
# #rfPredictions=gridCV.predict(X_test)
# #pred_to_csv(rfPredictions,'./rfsub.csv')



In [ ]:
# print(gridCV.best_score_)
# n_estimators=gridCV.best_params_['n_estimators']
# n_estimators
# n_estimators=1000

In [ ]:
#GBR=ensemble.GradientBoostingRegressor(criterion= 'mae',verbose=1)
#GBR.fit(X_train,Y_train)
#GBRpred=GBR.predict(X_test)
#pred_to_csv(GBRpred,"./GBRpred.csv")
## add elevation.nc file's control var to ens1. and perb to other ensemblers
#param_grid={'learning_rate':[0.05,0.1],'n_estimators':[100,500],'max_depth':[8],'criterion': ['mae']}

In [ ]:
#xg_reg = xgb.XGBRegressor()
#xg_reg.fit(X_train,Y_train)
#pred_to_csv(xg_reg.predict(X_test),'./xgbpred.csv')

In [ ]:

#xg_reg = xgb.XGBRegressor(objective ='reg:linear', colsample_bytree = 0.3, learning_rate = 0.1, alpha = 10, n_estimators = 500)
#xg_reg.fit(X_train,Y_train)
#pred_to_csv(xg_reg.predict(X_test),'./xgbpred2.csv')

In [ ]:
#best so far
#xg_reg = xgb.XGBRegressor(objective ='reg:linear',eval_metric='mae', colsample_bytree = 0.7,subsample=1,max_depth=10 , learning_rate = 0.1, alpha = 10, n_estimators = 500)
#xg_reg.fit(X_train,Y_train)
#pred_to_csv(xg_reg.predict(X_test),'./xgbpred2.csv')

In [ ]:
# xg_reg=xgb.XGBRegressor(objective='reg:linear',eval_metric='mae')
# params={'colsample_bytree' : [0.5,0.7,1],'subsample':[0.5,1],'max_depth':[None,5,10] , 'learning_rate' : [0.01,0.1], 'alpha': [5,10], 'n_estimators': [500,1000]}
# xggrid = GridSearchCV(xg_reg, n_jobs=-1,param_grid=params, cv=5,scoring=eval_metric,verbose=1)
# xggrid.fit(X_train,Y_train)
# print(xggrid.best_score_)
# pred_to_csv(xggrid.predict(X_test),'./xgbtuned2.csv')

In [ ]:
xg_reg = xgb.XGBRegressor(objective ='reg:linear',eval_metric='mae', colsample_bytree = 0.7,subsample=1,max_depth=10 , learning_rate = 0.01, alpha = 1, n_estimators = 1000)
xg_reg.fit(X_train,Y_train)
pred=xg_reg.predict(X_test)
pred_to_csv(pred,'./xgpred2.csv')

In [ ]:
# svr=SVR(kernel='linear',C=40)
# ridgemodel = RidgeCV(alphas=np.logspace(-3, 3, 10,base=10), cv=5,scoring=eval_metric)
# rfmodel=ensemble.RandomForestRegressor(n_estimators=1000)
# xg_reg = xgb.XGBRegressor(objective ='reg:linear',eval_metric='mae', colsample_bytree = 0.7,subsample=1,max_depth=10 , learning_rate = 0.01, alpha = 1, n_estimators = 1000)


In [ ]:
# knn=KNeighborsRegressor()
# params={'n_neighbors':[5,10,50]}
# knngrid=GridSearchCV(knn,param_grid=params,cv=5)
# knngrid.fit(X_train,Y_train)
# pred=knngrid.predict(X_test)
# pred_to_csv(pred,'./knnpred.csv')